# Exploratoy analysis of unkown Gaia DR2 dataset

In [7]:
IFrame('https://upload.wikimedia.org/wikipedia/en/0/01/Gaia_spacecraft.jpg', width=600, height=300)

#### Context

I have received a dataset with some entries of an unknown field with Gaia DR2 parameters. I will explore the data set and try to gain some insight on what science can be extracted from the data. The file is in csv format, so I will load it with pandas for exploratory analysis.

First I will load all the required modules

In [8]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = (16.0, 8.0)


Now I define some global variables

# Download and load the data
I will use a bash command to download the data and then untar the file. We can do it with bash (nice to show how to use bash code in the notebook), or with python, so we can manage the files in a general way.

In [66]:
%%bash
wget -q http://www.jb.man.ac.uk/~jmoldon/gaia_data.tgz
tar xvfz gaia_data.tgz
#ls -lh gaia_data.csv

gaia_data.csv


In [62]:
download_link = 'http://www.jb.man.ac.uk/~jmoldon/gaia_data.tgz'

In [65]:
def download_file(download_link, force_download=False):
    """
    Use curl to follow a link 'download_link' to get a file 'filename'
    Will not be downloaded if the file already exists
    """
    # Get the name of the file to be downloaded
    filename = os.path.split(download_link)[-1]
    # Check if the file exists and download it otherwise
    if not os.path.isfile(filename) or force_download:
        command = f'wget {download_link}'
        print('Downloading file with command: {}'.format(command))
        os.system(command)
        print(f'Check file is downloaded: {os.path.isfile(filename)}')
    else:
        print('File was already downloaded. Will not download it again')
    return filename

def untar_file(filename):
    command = 'tar xvfz {}'.format(filename)
    print('Running: {}'.format(command))
    os.system(command)
    print('Done')
    
downloaded_file = download_file(download_link)
untar_file(downloaded_file)

Check file is downloaded: True
Running: tar xvfz gaia_data.tgz
Done
